In [1]:
import os
import time
import pathlib
from concurrent.futures import ThreadPoolExecutor

from instagrapi import Client

In [2]:
def to_jpg(r: pathlib.WindowsPath):
    if isinstance(r, pathlib.WindowsPath):
        r.rename(r.with_suffix('.jpg'))
    else:
        raise ValueError

In [3]:
timeout = 4
batch_size = 16
executor = ThreadPoolExecutor(max_workers=batch_size)

In [7]:
cl = Client()
cl.login(os.environ.get("InstaLogin"), os.environ.get("InstaPassword"), relogin=True)

Status 429: Too many requests
Ignore 429: Continue login


True

In [ ]:
user_name = ""
user_id = cl.user_id_from_username(user_name)
save_path = pathlib.Path(f'markup/{user_name}/')
save_path.mkdir(exist_ok=True)

In [ ]:
medias = cl.user_medias(user_id, 100)

In [ ]:
num_batches = len(medias)//batch_size + int(len(medias)%batch_size != 0)
for b in range(num_batches):
    time_start = time.time()
    
    tasks = []
    for media in medias[b*batch_size:(b+1)*batch_size]:
        if media.media_type == 1:
            task =  executor.submit(cl.photo_download, media.pk, folder=save_path)
        elif media.media_type == 8:
            task = executor.submit(cl.album_download, media.pk, folder=save_path)
        tasks.append(task)
        
        
    for r in [t.result() for t in tasks]:
        if isinstance(r, list):
            for _r in r:
                to_jpg(_r)
        else:
            to_jpg(r)
        
        
    passed_time = time.time() - time_start
    if passed_time < timeout:
        time.sleep(timeout - passed_time)